# Title


## Import Libraries and Data
See prepare-data.ipynb for detail on data prep

In [1]:
#import libs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import random
warnings.filterwarnings('ignore')
from datetime import datetime
import statsmodels.formula.api as smf

from sklearn.decomposition import PCA

In [ ]:
# read in clean data
airbnb_data_dummies = pd.read_csv('./data/clean/cleaned-listings-dummies.csv')
airbnb_data_dummies_bio = pd.read_csv('./data/clean/cleaned-listings-dummies-bio.csv')
airbnb_data_basic = pd.read_csv('./data/clean/cleaned-listings-base.csv')

agg_pop_words = pd.read_csv('./output.csv')
agg_pop_words.rename(index=str, columns={"Unnamed: 0": "id", "0": "top_word_counts"} , inplace=True)


In [ ]:
# drop nas
airbnb_data_dummies_bio = airbnb_data_dummies_bio.dropna()

In [ ]:
# get counts for all words and phrases

# scores = pd.read_csv('./all_counts.csv')
# scores.rename(index=str, columns={"Unnamed: 0": "phrase", "0": "counts"} , inplace=True)
# scores.sort_values(by=['counts'], ascending=False, inplace=True)

# scores

In [ ]:
# get counts for all phrase scores

# phrases_scores = pd.read_csv('./just-phrases.csv')
# phrases_scores.sort_values(by=['counts'], ascending=False, inplace=True)
# phrases_scores = phrases_scores[['phrase', 'counts']]

# phrases_scores

In [ ]:
# list(airbnb_data_dummies_bio.columns.values)
# airbnb_data_dummies_bio.bio_score


In [ ]:
# join top scores with full bio set

airbnb_data_dummies_bio = airbnb_data_dummies_bio.set_index('id')
agg_pop_words = agg_pop_words.set_index('id')

airbnb_data_dummies_bio = airbnb_data_dummies_bio.join(other=agg_pop_words, how='inner')

In [ ]:
# pca_red = PCA(n_components=10).fit_transform(airbnb_data_dummies_bio)

# pca_red.explained_variance_ratio_

In [ ]:
# large formula
formula = 'price ~  top_word_counts + host_response_rate + accommodates + guests_included + bathrooms + bedrooms + beds + number_of_reviews + review_scores_rating + review_scores_accuracy + review_scores_cleanliness + review_scores_checkin + review_scores_communication + review_scores_location + review_scores_value + description_score + bio_score + host_is_superhost + neighbourhood_Ballard  + neighbourhood_Cascade  + neighbourhood_Delridge + neighbourhood_Downtown + neighbourhood_Interbay + neighbourhood_Magnolia + neighbourhood_Northgate + property_type_Apartment  + property_type_Condominium + property_type_House + property_type_Loft + property_type_Townhouse + neighbourhood_Beacon_Hill + neighbourhood_Capitol_Hill + neighbourhood_Central_Area + neighbourhood_Lake_City + neighbourhood_Other_neighborhoods + neighbourhood_Queen_Anne + neighbourhood_Rainier_Valley + neighbourhood_Seward_Park + neighbourhood_University_District + neighbourhood_West_Seattle + property_type_Bed_Breakfast + room_type_Entire_home_apt + room_type_Private_room + room_type_Shared_room'

formula_price = 'price ~  top_word_counts + host_response_rate + accommodates + guests_included + bathrooms + bedrooms + number_of_reviews + review_scores_rating + review_scores_cleanliness + review_scores_location + review_scores_value + description_score + bio_score + host_is_superhost + neighbourhood_Ballard  + neighbourhood_Cascade  + neighbourhood_Delridge + neighbourhood_Downtown + neighbourhood_Magnolia + neighbourhood_Northgate + property_type_Apartment  + property_type_Condominium + property_type_House + property_type_Loft + property_type_Townhouse + neighbourhood_Beacon_Hill + neighbourhood_Capitol_Hill + neighbourhood_Central_Area + neighbourhood_Lake_City + neighbourhood_Other_neighborhoods + neighbourhood_Queen_Anne + neighbourhood_Rainier_Valley + neighbourhood_Seward_Park + neighbourhood_University_District + room_type_Entire_home_apt + room_type_Private_room + room_type_Shared_room'

formula_review = 'review_scores_rating ~  price + top_word_counts + host_response_rate + accommodates + guests_included + bathrooms + bedrooms + number_of_reviews + review_scores_cleanliness + review_scores_location + review_scores_value + description_score + bio_score + host_is_superhost + neighbourhood_Ballard  + neighbourhood_Cascade  + neighbourhood_Delridge + neighbourhood_Downtown + neighbourhood_Magnolia + neighbourhood_Northgate + property_type_Apartment  + property_type_Condominium + property_type_House + property_type_Loft + property_type_Townhouse + neighbourhood_Beacon_Hill + neighbourhood_Capitol_Hill + neighbourhood_Central_Area + neighbourhood_Lake_City + neighbourhood_Other_neighborhoods + neighbourhood_Queen_Anne + neighbourhood_Rainier_Valley + neighbourhood_Seward_Park + neighbourhood_University_District + room_type_Entire_home_apt + room_type_Private_room + room_type_Shared_room'

formula_slim = 'price ~  top_word_counts + host_response_rate + description_score + bio_score + host_is_superhost'

# fit ordinary least squares model
mod1 = smf.ols(formula=formula_price, data=airbnb_data_dummies_bio).fit()
mod1.summary()

# # fit ordinary least squares model
# mod2 = smf.ols(formula=formula_slim, data=airbnb_data_dummies_bio).fit()
# mod2.summary()

In [ ]:
plt.figure(figsize=(12,8))
plt.scatter(airbnb_data_dummies_bio['bio_score'], airbnb_data_dummies_bio['price'], alpha = .4)
plt.xlabel('Sentiment Score of Host Bio')
plt.ylabel('Price (USD)')
plt.title('Sentiment Score vs. Price')

In [ ]:
plt.figure(figsize=(12,8))

top_words_barplot = sns.barplot(x="top_word_counts", y="price", data=airbnb_data_dummies_bio)

top_words_barplot.set(xlabel='Count Of Top Words And Pharses In Host Bio', ylabel='Price (USD)',
                      title='Listing Price And Prevalence Of Select Phrases In Host Bio')

top_words_barplot